In [1]:
from preprocess import get_img_paths, prep_data
import numpy as np

### Get image file paths

In [2]:
fractured, nfractured = get_img_paths('train')

### Preprocess data

In [3]:
x_1, y_1 = prep_data(fractured, 1)
x_0, y_0 = prep_data(nfractured, 0)
x_train = np.concatenate([x_0,x_1], axis=0)
y_train = np.concatenate([y_0,y_1], axis=0)
x_train.shape, y_train.shape

((8538, 50176), (8538, 1))